In [1]:
# from pandas.core.indexes.base import InvalidIndexError
import io
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from matplotlib.font_manager import FontProperty
from sklearn.impute import SimpleImputer
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline
# font = FontProperties(fname='/Library/Fonts/Heiti.ttc')

In [2]:
# 收集数据
# 没有表格文件时自定义数据
csv_data = '''
《肖申克的救赎》,《控方证人》,《这个杀手不太冷》,《霸王别姬》,《美丽人生》,《阿甘正传》,《辛德勒的名单》,姓名
,4.0,,4.0,,5.0,,'刘一'
4.0,,5.0,3.0,5.0,,,'陈二'
3.0,4.0,,3.0,2.0,3.0,3.0,'张三'
2.0,3.0,,3.0,,,,'李四'
3.0,4.0,,5.0,3.0,3.0,,'王五'
,,4.0,,4.0,2.0,,'赵六'
3.0,,1.0,5.0,3.0,3.0,2.0,'孙七'
2.0,,2.0,,1.0,,,'周八'
1.0,2.0,,,,2.0,,'吴九'
,5.0,,4.0,,3.0,3.0,'郑十'
'''
if not os.path.exists('datasets/movie.xlsx'):
  # 将文件读入内存
  csv_data = io.StringIO(csv_data)
  df = pd.read_csv(csv_data, header=0)
  # df.index = df['姓名'].tolist()
else:
  # 从表格中获取数据
  df = pd.read_excel('datasets/movie.xlsx', header=0)
  # df.index = df['姓名'].tolist()
df

,《肖申克的救赎》,《控方证人》,《这个杀手不太冷》,《霸王别姬》,《美丽人生》,《阿甘正传》,《辛德勒的名单》,姓名
0,NaN,4.0,NaN,4.0,NaN,5.0,NaN,'刘一'
1,4.0,NaN,5.0,3.0,5.0,NaN,NaN,'陈二'
2,3.0,4.0,NaN,3.0,2.0,3.0,3.0,'张三'
3,2.0,3.0,NaN,3.0,NaN,NaN,NaN,'李四'
4,3.0,4.0,NaN,5.0,3.0,3.0,NaN,'王五'
5,NaN,NaN,4.0,NaN,4.0,2.0,NaN,'赵六'
6,3.0,NaN,1.0,5.0,3.0,3.0,2.0,'孙七'
7,2.0,NaN,2.0,NaN,1.0,NaN,NaN,'周八'
8,1.0,2.0,NaN,NaN,NaN,2.0,NaN,'吴九'
9,NaN,5.0,NaN,4.0,NaN,3.0,3.0,'郑十'


In [3]:
# 数据缺失值处理
def imputer_data(np_arr):
  # 中位数strategy=median，平均数strategy=mean，众数strategy=most_frequent
  imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
  # 找到用户没有评分的电影
  imr = imputer.fit(np_arr.iloc[:, :-1])
  # 将用户没有评分的电影评分用0填充
  imputed_df = imr.transform(np_arr.iloc[:, :-1])

  return imputed_df


imputed_df = imputer_data(df)
imputed_df

array([[0., 4., 0., 4., 0., 5., 0.],
       [4., 0., 5., 3., 5., 0., 0.],
       [3., 4., 0., 3., 2., 3., 3.],
       [2., 3., 0., 3., 0., 0., 0.],
       [3., 4., 0., 5., 3., 3., 0.],
       [0., 0., 4., 0., 4., 2., 0.],
       [3., 0., 1., 5., 3., 3., 2.],
       [2., 0., 2., 0., 1., 0., 0.],
       [1., 2., 0., 0., 0., 2., 0.],
       [0., 5., 0., 4., 0., 3., 3.]])

In [4]:
# 数据标准化前用户之间的相似度
user_1 = imputed_df[[0]]
user_2 = imputed_df[[1]]
user_8 = imputed_df[[7]]
user_9 = imputed_df[[8]]
print('电影名:{}'.format(df.columns[:-1].values))
print('刘一:{}'.format(user_1))
print('陈二:{}'.format(user_2))
print('周八:{}'.format(user_8))
print('吴九:{}'.format(user_9))

print('刘一和陈二的余弦相似度:{}'.format(cosine_similarity(user_1, user_2)))
print('陈二和周八的余弦相似度:{}'.format(cosine_similarity(user_2, user_8)))
print('周八和吴九的余弦相似度:{}'.format(cosine_similarity(user_8, user_9)))

电影名:['《肖申克的救赎》' '《控方证人》' '《这个杀手不太冷》' '《霸王别姬》' '《美丽人生》' '《阿甘正传》' '《辛德勒的名单》']
刘一:[[0. 4. 0. 4. 0. 5. 0.]]
陈二:[[4. 0. 5. 3. 5. 0. 0.]]
周八:[[2. 0. 2. 0. 1. 0. 0.]]
吴九:[[1. 2. 0. 0. 0. 2. 0.]]
刘一和陈二的余弦相似度:[[0.18353259]]
陈二和周八的余弦相似度:[[0.88527041]]
周八和吴九的余弦相似度:[[0.22222222]]


In [5]:
# 标准化评分
def nonzero_mean(np_arr):
  """计算矩阵内每一行不为0元素的平均数"""
  # 找到数组内评分不为0的即非0元素
  exist = (np_arr != 0)
  # 行非0元素总和
  arr_sum = np_arr.sum(axis=1)
  # 行非0元素总个数
  arr_num = exist.sum(axis=1)

  return arr_sum/arr_num


def standard_data(np_arr):
  standardized_df = np_arr.copy()
  # 非0元素行下标
  nonzero_rows = np.nonzero(np_arr)[0]
  # 非0元素列下标
  nonzero_columns = np.nonzero(np_arr)[1]
  # 非0元素行平均值
  nonzero_rows_mean = nonzero_mean(np_arr)
  # 遍历并修改所有非0元素
  for ind in range(len(nonzero_rows)):
    # 第ind个元素的行标和列标确定一个元素
    i = nonzero_rows[ind]
    j = nonzero_columns[ind]
    standardized_df[i, j] = round(np_arr[i, j]-nonzero_rows_mean[i], 2)

  return standardized_df


standardized_df = standard_data(imputed_df)

In [6]:
# 数据标准化后用户之间的相似度
user_1 = standardized_df[[0]]
user_2 = standardized_df[[1]]
user_8 = standardized_df[[7]]
user_9 = standardized_df[[8]]
print('电影名:{}'.format(df.columns[:-1].values))
print('刘一:{}'.format(user_1))
print('陈二:{}'.format(user_2))
print('周八:{}'.format(user_8))
print('吴九:{}'.format(user_9))

print('刘一和陈二的余弦相似度:{}'.format(cosine_similarity(user_1, user_2)))
print('陈二和周八的余弦相似度:{}'.format(cosine_similarity(user_2, user_8)))
print('周八和吴九的余弦相似度:{}'.format(cosine_similarity(user_8, user_9)))

电影名:['《肖申克的救赎》' '《控方证人》' '《这个杀手不太冷》' '《霸王别姬》' '《美丽人生》' '《阿甘正传》' '《辛德勒的名单》']
刘一:[[ 0.   -0.33  0.   -0.33  0.    0.67  0.  ]]
陈二:[[-0.25  0.    0.75 -1.25  0.75  0.    0.  ]]
周八:[[ 0.33  0.    0.33  0.   -0.67  0.    0.  ]]
吴九:[[-0.67  0.33  0.    0.    0.    0.33  0.  ]]
刘一和陈二的余弦相似度:[[0.30464382]]
陈二和周八的余弦相似度:[[-0.24925404]]
周八和吴九的余弦相似度:[[-0.33163342]]


In [7]:
# 预测
# 对新数据处理成np.array数组
new_user = '''
1,2,,2,2,2,,王二麻子
2,1,2,,5,1,4,烂谷子
1,2,2,2,3,,4,大芝麻
'''


def predict(new_user, similarity_tool='cosine_similarity'):
  rating_list = []
  movie_list = []

  # 对于输入数据为或str和numpy数组做不同的处理
  if isinstance(new_user, str):
    new_df = pd.read_csv(io.StringIO(new_user), header=None)
  else:
    new_df = pd.DataFrame(new_user)

  new_df.columns = ['《肖申克的救赎》', '《控方证人》', '《这个杀手不太冷》', '《霸王别姬》', '《美丽人生》', '《阿甘正传》', '《辛德勒的名单》', '姓名']

  # 填充数据并对数据进行标准化
  imputed_new_df = imputer_data(new_df)
  standardized_new_user = standard_data(imputed_new_df)

  # 通过余弦相似度计算预测用户与已有样本之间的相似度
  user_similarity_list = []
  for ind in range(len(standardized_df)):
    user = standardized_df[[ind]]
    mod = sys.modules['__main__']
    file = getattr(mod, similarity_tool, None)
    user_similarity_list.append(cosine_similarity(user, standardized_new_user)[0])

  # 将余弦相似度列表构造成numpy数组方便计算
  users_similarity_arr = np.array(user_similarity_list).reshape(standardized_df.shape[0], standardized_new_user.shape[0])

  # 遍历所有用户对她没有评分的电影计算预测评分值
  for ind in range(len(new_df)):
    empty_rating_ind = []
    # 获取名字信息
    name = new_df['姓名'][ind]
    nonzero_list = np.nonzero(imputed_new_df[[ind], :])[1]
    user_similarity_arr = users_similarity_arr[:, [ind]].reshape(1, -1)

    # 找到预测用户没有评分电影的索引
    for j in range(standardized_new_user.shape[1]):
      if j not in nonzero_list:
        empty_rating_ind.append(j)

    # 遍历预测用户没有评分的电影计算预测评分值
    for rating_ind in empty_rating_ind:
      # 计算用户的加权评分总和
      user_rating_list = imputed_df[:, rating_ind].reshape(1, -1)
      rating_arr = user_similarity_arr*user_rating_list
      rating_sum = rating_arr.sum(axis=1)
      # 计算用户的总相似度
      user_similarity_sum = user_similarity_arr.sum(axis=1)

      # 当用户总相似度为0时打印提示消息
      if rating_sum == 0:
        print('亲，{}不怎么看电影我实在无能为力！'.format(name))
      else:
        rating = rating_sum/user_similarity_sum
        print('*{}*可能会给电影{}评分{}'.format(name, df.columns[:-1].values[rating_ind], round(rating[0], 2)))
      # 统计评分
      rating_list.append(round(rating[0], 2))

    empty_rating_ind = []

  return rating_list


rating_list = predict(new_user)

*王二麻子*可能会给电影《这个杀手不太冷》评分-0.52
*王二麻子*可能会给电影《辛德勒的名单》评分0.31
*烂谷子*可能会给电影《霸王别姬》评分2.87
*大芝麻*可能会给电影《阿甘正传》评分4.41


In [8]:
l = np.random.randint(0, 6, size=(50, 8))
l[:, [-1]] = 0

rating_list = predict(l)

*0*可能会给电影《霸王别姬》评分2.47
*0*可能会给电影《霸王别姬》评分1.47
*0*可能会给电影《肖申克的救赎》评分1.88
*0*可能会给电影《肖申克的救赎》评分1.34
*0*可能会给电影《阿甘正传》评分-0.08
*0*可能会给电影《阿甘正传》评分3.26
*0*可能会给电影《辛德勒的名单》评分2.1
亲，0不怎么看电影我实在无能为力！
亲，0不怎么看电影我实在无能为力！
亲，0不怎么看电影我实在无能为力！
*0*可能会给电影《这个杀手不太冷》评分-2.47
*0*可能会给电影《辛德勒的名单》评分5.93
亲，0不怎么看电影我实在无能为力！
*0*可能会给电影《霸王别姬》评分2.8
*0*可能会给电影《辛德勒的名单》评分2.1
*0*可能会给电影《控方证人》评分11.79
*0*可能会给电影《阿甘正传》评分3.39
*0*可能会给电影《控方证人》评分2.38
*0*可能会给电影《阿甘正传》评分1.06
*0*可能会给电影《控方证人》评分3.48
*0*可能会给电影《这个杀手不太冷》评分-1.4
*0*可能会给电影《阿甘正传》评分2.52
*0*可能会给电影《阿甘正传》评分1.51
*0*可能会给电影《辛德勒的名单》评分1.02
*0*可能会给电影《美丽人生》评分-0.38
*0*可能会给电影《阿甘正传》评分4.35
*0*可能会给电影《控方证人》评分4.42
*0*可能会给电影《霸王别姬》评分2.29
*0*可能会给电影《肖申克的救赎》评分3.78
*0*可能会给电影《霸王别姬》评分3.59
亲，0不怎么看电影我实在无能为力！
亲，0不怎么看电影我实在无能为力！
亲，0不怎么看电影我实在无能为力！
*0*可能会给电影《肖申克的救赎》评分2.24
*0*可能会给电影《阿甘正传》评分1.97
*0*可能会给电影《辛德勒的名单》评分2.11
*0*可能会给电影《肖申克的救赎》评分1.98
*0*可能会给电影《这个杀手不太冷》评分0.92
*0*可能会给电影《美丽人生》评分3.03
*0*可能会给电影《美丽人生》评分1.0
*0*可能会给电影《辛德勒的名单》评分1.78
*0*可能会给电影《控方证人》评分2.43
*0*可能会给电影《这个杀手不太冷》评分0.13
*0*可能会给电影《美丽人生》评分3.95
*0*可能会给电影《控方证人》评分-1.52
*0*